# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

ANSWER: In this problem, we are trying to differentiate students into a discrete number classes (those who need early intervention and those who do not).  Therefore, this is a classification problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [49]:
# Import libraries
import numpy as np
import pandas as pd
import os
from sklearn.cross_validation import train_test_split

In [51]:
# Read student data
# I loaded the data this way because the suggested solution does not work on my machine
fname = os.path.join(os.getcwd(), 'student_intervention_system/student-data.csv')
student_data = pd.read_csv(fname)
print str.format("Student data read successfully from: {}", fname)
# Note: The last column 'passed' is the target/label, all other are feature columns

IOError: File student-data.csv does not exist

Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]
n_passed = len([val for val in student_data['passed'] if val == "yes"]) 
n_failed = n_students - n_passed 
grad_rate = n_passed.__float__()/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

student_data_shuffled = student_data.sample(frac=1)
X_all = student_data_shuffled[feature_cols]  # feature values for all students
y_all = student_data_shuffled[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
126     GP   F   15       U     LE3       A     3     4     other     other   
82      GP   F   15       U     LE3       T     3     2  services     other   
337     GP   F   17       U     GT3       T     3     2     other     other   
62      GP   F   16       U     LE3       T     1     2     other  services   
34      GP   M   16       U     GT3       T     3     2     other     other   

      ...    higher internet  romantic  famrel  freetime goout Dalc Walc  \
126   ...       yes      yes       yes       5         3     2    1    1   
82 

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train = X_all[0:num_train]
y_train = y_all[0:num_train]
X_test = X_all[num_train:]
y_test = y_all[num_train:]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [53]:
# Train a model
import time
import copy

def train_classifier(clf, X_train, y_train, training_set_size=len(X_train)):
    print "Training {} with {} data points.".format(clf.__class__.__name__, training_set_size)
    input_X = X_train[0:training_set_size]
    input_Y = y_train[0:training_set_size]
    start = time.time()
    clf.fit(input_X, input_Y)
    end = time.time()
    training_time = end - start 
    print "Done!\nTraining time (secs): {:.4f}".format(training_time)
    return training_time

# TODO: Choose a model, import it and instantiate an object
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
models = {
    'dt': DecisionTreeClassifier(),
    'lr': LogisticRegression(),
    'rf': RandomForestClassifier(),
}

trained_classifiers = {}
TRAINING_SIZES = [100, 200, 300]
for x in TRAINING_SIZES:
    trained_classifiers[x] = [] 
# Fit model to training data
for (model_name, model) in models.items():
    for training_size in TRAINING_SIZES: 
        clf = copy.deepcopy(model)
        training_time = train_classifier(clf, X_train, y_train, training_size)
        new_model_name = model_name + str(training_size)
        
        trained_classifiers[training_size].append(clf)
        
        #print trained_classifiers.get(new_model_name).get("training_time")  # you can inspect the learned model by printing it

Training DecisionTreeClassifier with 100 data points.
Done!
Training time (secs): 0.0012
Training DecisionTreeClassifier with 200 data points.
Done!
Training time (secs): 0.0034
Training DecisionTreeClassifier with 300 data points.
Done!
Training time (secs): 0.0021
Training LogisticRegression with 100 data points.
Done!
Training time (secs): 0.0011
Training LogisticRegression with 200 data points.
Done!
Training time (secs): 0.0015
Training LogisticRegression with 300 data points.
Done!
Training time (secs): 0.0032
Training RandomForestClassifier with 100 data points.
Done!
Training time (secs): 0.0231
Training RandomForestClassifier with 200 data points.
Done!
Training time (secs): 0.0216


Training RandomForestClassifier with 300 data points.
Done!
Training time (secs): 0.0236


In [47]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target, num_training_examples=None):
    if num_training_examples == None:
        num_training_examples=len(target)
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features[0:num_training_examples])
    end = time.time()
    prediction_time = end - start
    print "Done!\nPrediction time (secs): {:.3f}".format(training_time)
    return f1_score(target.values[0:num_training_examples], y_pred, pos_label='yes'), prediction_time

for training_set_size in trained_classifiers.keys():
    for model in trained_classifiers[training_set_size]:
        train_f1_score, prediction_time = predict_labels(model, X_train, y_train, training_set_size)
        print "F1 score for training set: {}".format(train_f1_score)
        print "Preciction time for training set of size {}: {}".format(training_set_size, prediction_time)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.025
F1 score for training set: 1.0
Preciction time for training set of size 200: 0.00101113319397
Predicting labels using SVC...
Done!
Prediction time (secs): 0.025
F1 score for training set: 0.879194630872
Preciction time for training set of size 200: 0.00327706336975
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.025
F1 score for training set: 0.988593155894
Preciction time for training set of size 200: 0.00172305107117
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.025
F1 score for training set: 1.0
Preciction time for training set of size 300: 0.000586032867432
Predicting labels using SVC...
Done!
Prediction time (secs): 0.025
F1 score for training set: 0.864628820961
Preciction time for training set of size 300: 0.00539517402649
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.025
F1 score for train

In [48]:
# Predict on test data
for training_set_size in trained_classifiers.keys():
    for model in trained_classifiers[training_set_size]:
        test_f1_score, prediction_time = predict_labels(model, X_test, y_test)
        print "F1 score for model {} on test set: {}".format(model.__class__.__name__, test_f1_score)
        print "Prediction time for model {} trained on test set size {}: {}".format(
            model.__class__.__name__, training_set_size, prediction_time
        )

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.025
F1 score for model DecisionTreeClassifier on test set: 0.638655462185
Prediction time for model DecisionTreeClassifier trained on test set size 200: 0.000761985778809
Predicting labels using SVC...
Done!
Prediction time (secs): 0.025
F1 score for model SVC on test set: 0.773333333333
Prediction time for model SVC trained on test set size 200: 0.00153398513794
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.025
F1 score for model RandomForestClassifier on test set: 0.704
Prediction time for model RandomForestClassifier trained on test set size 200: 0.0015561580658
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.025
F1 score for model DecisionTreeClassifier on test set: 0.708661417323
Prediction time for model DecisionTreeClassifier trained on test set size 300: 0.00046706199646
Predicting labels using SVC...
Done!
Prediction time (secs

In [41]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [11]:
# TODO: Train and predict using two other models

# Already done -- did this by iterating through the dictionary of models above.

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [54]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, pos_label="yes")

random_forest_optimization_params = {'max_depth': [i for i in xrange(1,10)],
              'criterion':("gini","entropy"),
              'max_features':("auto","sqrt","log2"),
              "bootstrap": [True, False],
              'min_samples_leaf': [i for i in xrange(1,5)],
              'min_samples_split': [2*i for i in xrange(1,4)],
              }
decision_tree_optimization_params = {'max_depth': [i for i in xrange(1,10)],
              'criterion':("gini","entropy"),
              'max_features':("auto","sqrt","log2"),
              'min_samples_leaf': [i for i in xrange(1,5)],
              'min_samples_split': [2*i for i in xrange(1,4)],
              }
params_lr = {
              'dual': [False, True],
              'fit_intercept':[False, True],
              'random_state':[None, 42],
              'warm_start': [True, False],
              'n_jobs' : [1,2,3],
              'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}
optimized_classifiers = {
    "dt_randomized": {"clf": RandomizedSearchCV(DecisionTreeClassifier(), decision_tree_optimization_params, n_iter=100, scoring=f1_scorer)},
    "dt_grid": {"clf": GridSearchCV(DecisionTreeClassifier(), decision_tree_optimization_params)},
    "rf_randomized": {"clf": RandomizedSearchCV(RandomForestClassifier(), random_forest_optimization_params, n_iter=100, scoring=f1_scorer)},
    "rf_grid": {"clf": GridSearchCV(RandomForestClassifier(), random_forest_optimization_params, scoring=f1_scorer)},
    "lr_randomized": {"clf": RandomizedSearchCV(LogisticRegression(), params_lr, n_iter=5, scoring=f1_scorer)},
    "lr_grid": {"clf": GridSearchCV(LogisticRegression(), params_lr, scoring=f1_scorer)}
}


for model in optimized_classifiers.values():
    training_time = train_classifier(model.get("clf"), X_train, y_train)
    model["training_time"] = training_time
    print model.get("clf")  
    train_f1_score, prediction_time = predict_labels(model.get("clf"), X_train, y_train)
    
    print "F1 score for training set: {}".format(train_f1_score)
    model["train_f1_score"]  = train_f1_score
    model["prediction_time_training"] = prediction_time

    test_f1_score, prediction_time = predict_labels(model.get("clf"), X_test, y_test)
    print "F1 score for test set: {}".format(test_f1_score)
    model['prediction_time_test'] = prediction_time

Training GridSearchCV with 300 data points.
Done!
Training time (secs): 106.8533


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 3, 4], 'min_samples_split': [2, 4, 6], 'criterion': ('gini', 'entropy'), 'max_features': ('auto', 'sqrt', 'log2'), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 106.853
F1 score for training set: 0.822085889571
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 106.853
F


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 3, 4], 'min_samples_split': [2, 4, 6], 'criterion': ('gini', 'entropy'), 'max_features': ('auto', 'sqrt', 'log2'), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Predicting labels using RandomizedSearchCV...
Done!
Prediction time (secs): 8.275
F1 score for training set: 0.845665961945
Predicting lab


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'warm_start': [True, False], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'n_jobs': [1, 2, 3], 'fit_intercept': [False, True], 'random_state': [None, 42], 'dual': [False, True]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 6.777
F1 score for training set: 0.828157349896
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 6.777
F1 score for test set: 0.794871794872
Training RandomizedSearchCV with 300 data points.
Done!
Training time (secs): 0.1184


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'warm_start': [True, False], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'n_jobs': [1, 2, 3], 'fit_intercept': [False, True], 'random_state': [None, 42], 'dual': [False, True]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Predicting labels using RandomizedSearchCV...
Done!
Prediction time (secs): 0.118
F1 score for training set: 0.828157349896
Predicting labels using RandomizedSearchCV...
Done!
Prediction time (secs): 0.118
F1 score for test set: 0.794871794872
Training RandomizedSearc


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'max_features': ('auto', 'sqrt', 'log2'), 'min_samples_split': [2, 4, 6], 'criterion': ('gini', 'entropy'), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'min_samples_leaf': [1, 2, 3, 4]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Predicting labels using RandomizedSearchCV...
Done!
Prediction time (secs): 1.447
F1 score for training set: 0.813765182186
Predicting labels using RandomizedSearchCV...
Done!
Prediction time (secs): 1.447
F1 score for test set: 0.794871794872
Train


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': ('auto', 'sqrt', 'log2'), 'min_samples_split': [2, 4, 6], 'criterion': ('gini', 'entropy'), 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'min_samples_leaf': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 8.004
F1 score for training set: 0.814004376368
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 8.004
F1 score for test set: 0.739726027397
